In [1]:
import pandas as pd
import json


In [2]:
df_test = pd.read_csv('test.csv')

In [ ]:
df_test["label"].unique()

In [ ]:
df_test = df_test[df_test["annotation_type"] == "Rectangle"][ df_test["label"] == "riktning_symbol"]

In [ ]:
df_test

In [7]:
#data loader for the dataset
import torch
from PIL import Image
from torchvision.transforms import functional as F
import os
import cv2

class KBABygglovDataset(torch.utils.data.Dataset):
    def __init__(self, df, scale = 4):
        self.df = df
        self.transforms = None
        self.scale = scale
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = cv2.imread(row["image"])
        #contour = find_bounding_box(image)
        
        
        # Normalize bounding box coordinates
        width, height = row["original_width"], row["original_height"]
        target = {}

        x1 = row['x'] * width //100 
        y1 = row['y'] * height //100
        width = row['width'] * width //100
        height = row['height'] * height //100
        target["boxes"] = torch.as_tensor([[x1,y1, x1 + width, y1 + height]], 
                                dtype = torch.float32)
        target["labels"]=torch.as_tensor([1],
                        dtype = torch.int64)
        target["image_id"] = torch.as_tensor(idx)
        #scale the image and target to half size
        image = cv2.resize(image, (image.shape[1]// self.scale,  image.shape[0]// self.scale))

        target["boxes"] = target["boxes"]/ self.scale

        image = F.to_tensor(image)
        #cropped_image, target = crop_to_contour(image, target, contour)
        return image, target
        return None




test_dataset = KBABygglovDataset(df_test)

In [8]:
# Collate image-target pairs into a tuple.
def collate_fn(batch):
    return tuple(zip(*batch))
# Create the DataLoaders from the Datasets. 

'''val_dl = torch.utils.data.DataLoader(val_ds, 
                             batch_size = 4, 
                            shuffle = False, 
                    collate_fn = collate_fn)'''
test_dl = torch.utils.data.DataLoader(test_dataset, 
                               batch_size = 1, 
                              shuffle = False, 
                      collate_fn = collate_fn)

In [9]:
import torch

In [10]:
model = torch.load('riktning.model')

In [11]:
def unbatch(batch, device):
    """
    Unbatches a batch of data from the Dataloader.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        X: list
            List of images.
        y: list
            List of dictionaries.
    """
    X, y = batch
    X = [x.to(device) for x in X]
    y = [{k: v.to(device) for k, v in t.items()} for t in y]
    return X, y
def train_batch(batch, model, optimizer, device):
    """
    Uses back propagation to train a model.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        model: torch model
        optimizer: torch optimizer
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        loss: float
            Sum of the batch losses.
        losses: dict
            Dictionary containing the individual losses.
    """
    model.train()
    X, y = unbatch(batch, device = device)
    optimizer.zero_grad()
    losses = model(X, y)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    return loss, losses
@torch.no_grad()
def validate_batch(batch, model, optimizer, device):
    """
    Evaluates a model's loss value using validation data.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        model: torch model
        optimizer: torch optimizer
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        loss: float
            Sum of the batch losses.
        losses: dict
            Dictionary containing the individual losses.
    """
    model.train()
    X, y = unbatch(batch, device = device)
    optimizer.zero_grad()
    losses = model(X, y)
    loss = sum(loss for loss in losses.values())
    return loss, losses

In [12]:
@torch.no_grad()
def predict_batch(batch, model, device):
    model.to(device)
    model.eval()
    X, _ = unbatch(batch, device = device)
    predictions = model(X)
    return [x.cpu() for x in X], predictions
def predict(model, data_loader, device = "cpu"):
    images = []
    predictions = []
    for i, batch in enumerate(data_loader):
        X, p = predict_batch(batch, model, device)
        images.append(X)
        predictions.append(p)
    
    return images, predictions

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
images, predictions = predict(model, test_dl, device = device)

In [ ]:
device

In [ ]:
predictions[0][0]

In [16]:
import torchvision
def decode_prediction(prediction, 
                      score_threshold = 0.8, 
                      nms_iou_threshold = 0.2):
    """
    Inputs
        prediction: dict
        score_threshold: float
        nms_iou_threshold: float
    Returns
        prediction: tuple
    """
    boxes = prediction["boxes"]
    scores = prediction["scores"]
    labels = prediction["labels"]
    # Remove any low-score predictions.
    if score_threshold is not None:
        want = scores > score_threshold
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]
    # Remove any overlapping bounding boxes using NMS.
    if nms_iou_threshold is not None:
        want = torchvision.ops.nms(boxes = boxes, scores = scores, 
                                iou_threshold = nms_iou_threshold)
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]
    return {"boxes": boxes, 
            "labels": labels, 
            "scores": scores}

In [17]:
from Object_detection.object_detection_helper import visualize_image_with_bounding_boxes_colored_img

In [ ]:
for i in range(len(images)):
    print(i)
    new_pred = decode_prediction(predictions[i][0], score_threshold = 0.5)
    print(new_pred)
    visualize_image_with_bounding_boxes_colored_img(images[i][0], new_pred)

In [ ]:
negative_examples = pd.read_csv("annotations.csv")
negative_examples = negative_examples[negative_examples["annotation_type"] == "Rectangle"][ negative_examples["label"] == "riktning_saknas"]
negative_examples

In [20]:
#data loader for the dataset
import torch
from PIL import Image
from torchvision.transforms import functional as F
import os

class KBABygglovDataset_false(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.transforms = None
        print(len(self.df))
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = self.df.iloc[idx]["image"]
        image = Image.open(image_path)
        image = F.to_tensor(image)
        
        target = {}
        target["id"] = torch.as_tensor(row["annotation_id"])
        
        return image, target
  

In [ ]:
dataset_false = KBABygglovDataset_false(negative_examples)
false_dl = torch.utils.data.DataLoader(dataset_false, 
                                 batch_size =1, 
                                 shuffle = False, 
                        collate_fn = collate_fn)
images, predictions = predict(model, false_dl, device = device)

In [ ]:
for i in range(len(images)):
    print(i)
    new_pred = decode_prediction(predictions[i][0], score_threshold = 0.5)
    print(new_pred)
    visualize_image_with_bounding_boxes_colored_img(images[i][0], new_pred)
